# Simple Linear Regression


#### Loading modules.

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split


In [18]:
#Reading the csv file.
salary = pd.read_csv('resources/salary.csv')
salary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514 entries, 0 to 513
Data columns (total 11 columns):
salary         513 non-null float64
exprior        514 non-null int64
yearsworked    514 non-null int64
yearsrank      514 non-null int64
market         514 non-null float64
degree         514 non-null int64
otherqual      514 non-null int64
position       514 non-null int64
male           514 non-null int64
Field          514 non-null int64
yearsabs       514 non-null int64
dtypes: float64(2), int64(9)
memory usage: 44.3 KB


In [19]:
# Filling my NaN value with the mean of the column
# salary = salary.apply(lambda x: x.fillna(x.mean()))

In [20]:
salary.head()

,salary,exprior,yearsworked,yearsrank,market,degree,otherqual,position,male,Field,yearsabs
0,53000.0,0,0,0,1.17,1,0,1,1,3,0
1,58000.0,1,0,0,1.24,1,0,1,1,2,0
2,45500.0,0,0,0,1.21,1,0,1,1,3,2
3,35782.0,0,2,1,0.99,1,0,1,1,4,1
4,34731.0,0,2,2,0.91,1,0,1,1,4,1


In [24]:

# Creating the target variable for the test and training sets of data
X = salary.drop('salary', axis = 1)
y = salary['salary']
X.info()
# Splitting the data into test and training sets by 75/25
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.75, random_state = 42)
#print(X_train.shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514 entries, 0 to 513
Data columns (total 10 columns):
exprior        514 non-null int64
yearsworked    514 non-null int64
yearsrank      514 non-null int64
market         514 non-null float64
degree         514 non-null int64
otherqual      514 non-null int64
position       514 non-null int64
male           514 non-null int64
Field          514 non-null int64
yearsabs       514 non-null int64
dtypes: float64(1), int64(9)
memory usage: 40.3 KB


In [22]:
missing_values = X_train.isnull().sum().sum()
print('Number of missing values:',missing_values)

highest =  X_train.max()
print('The highest value is: ', highest)

Number of missing values: 0
The highest value is:  exprior         25.00
yearsworked     41.00
yearsrank       28.00
market           1.33
degree           1.00
otherqual        1.00
position         3.00
male             1.00
Field            4.00
yearsabs       118.00
dtype: float64
